## Model training and tuning for the healthcare sector

In [31]:
import pandas as pd
import numpy as np

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel

In [14]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_healthcare = df[df['GICS Sector'] == 'Health Care'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

# lag volume, high, low by 1 day
# df_healthcare[['volume', 'high', 'low']] = (
#     df_healthcare.groupby('ticker')[['volume', 'high', 'low']].shift(1)
# )
# df_healthcare = df_healthcare.dropna().reset_index(drop=True)

In [15]:
data = {}

for ticker, group in df_healthcare.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B'
    )

    # series = series.add_datetime_attribute('day') \
    #     .add_datetime_attribute('month') \
    #     .add_datetime_attribute('year') \
    #     .add_datetime_attribute('dayofweek') \
    #     .add_datetime_attribute('dayofyear') \
    #     .add_datetime_attribute('weekofyear') \
    #     .add_datetime_attribute('quarter')
    
    data[ticker] = series

In [16]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [ ]:
import time
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
        # future_covariates=train[ticker][['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']]
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for A
Model for A trained in 0.30 seconds
**************************************************
Training model for ABBV
Model for ABBV trained in 0.40 seconds
**************************************************
Training model for ABT
Model for ABT trained in 0.36 seconds
**************************************************
Training model for ALGN
Model for ALGN trained in 0.29 seconds
**************************************************
Training model for AMGN
Model for AMGN trained in 0.29 seconds
**************************************************
Training model for BAX
Model for BAX trained in 0.35 seconds
**************************************************
Training model for BDX
Model for BDX trained in 0.27 seconds
**************************************************
Training model for BIIB
Model for BIIB trained in 0.37 seconds
**************************************************
Training model for BMY
Model for BMY trained in 0.31 seconds
***************************************

In [18]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: A, MAPE: 4.10, MAE: 5.33
**************************************************
Ticker: ABBV, MAPE: 8.34, MAE: 15.81
**************************************************
Ticker: ABT, MAPE: 2.70, MAE: 3.26
**************************************************
Ticker: ALGN, MAPE: 5.17, MAE: 11.60
**************************************************
Ticker: AMGN, MAPE: 6.07, MAE: 18.82
**************************************************
Ticker: BAX, MAPE: 4.93, MAE: 1.67
**************************************************
Ticker: BDX, MAPE: 4.02, MAE: 8.78
**************************************************
Ticker: BIIB, MAPE: 32.97, MAE: 49.37
**************************************************
Ticker: BMY, MAPE: 6.48, MAE: 3.53
**************************************************
Ticker: BSX, MAPE: 18.39, MAE: 17.28
**************************************************
Ticker: CAH, MAPE: 9.83, MAE: 11.87
**************************************************
Ticker: CI, MAPE: 2.38, MAE: 7.45
**********

In [20]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(7.3685237134749375, 20.384451794129127)

## Prophet model

In [22]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for A


17:03:45 - cmdstanpy - INFO - Chain [1] start processing
17:03:48 - cmdstanpy - INFO - Chain [1] done processing


Model for A trained in 2.80 seconds
**************************************************
Training model for ABBV


17:03:48 - cmdstanpy - INFO - Chain [1] start processing
17:03:51 - cmdstanpy - INFO - Chain [1] done processing


Model for ABBV trained in 3.22 seconds
**************************************************
Training model for ABT


...

Model for WST trained in 3.08 seconds
**************************************************
Training model for ZBH


17:06:19 - cmdstanpy - INFO - Chain [1] start processing
17:06:22 - cmdstanpy - INFO - Chain [1] done processing


Model for ZBH trained in 3.15 seconds
**************************************************
Training model for ZTS


17:06:23 - cmdstanpy - INFO - Chain [1] start processing
17:06:25 - cmdstanpy - INFO - Chain [1] done processing


Model for ZTS trained in 3.34 seconds
**************************************************


In [23]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: A, MAPE: 18.08, MAE: 24.75
**************************************************
Ticker: ABBV, MAPE: 14.20, MAE: 26.41
**************************************************
Ticker: ABT, MAPE: 9.73, MAE: 11.76
**************************************************
Ticker: ALGN, MAPE: 30.36, MAE: 67.21
**************************************************
Ticker: AMGN, MAPE: 12.17, MAE: 36.99
**************************************************
Ticker: BAX, MAPE: 14.35, MAE: 4.88
**************************************************
Ticker: BDX, MAPE: 8.64, MAE: 19.80
**************************************************
Ticker: BIIB, MAPE: 87.30, MAE: 136.68
**************************************************
Ticker: BMY, MAPE: 28.48, MAE: 15.34
**************************************************
Ticker: BSX, MAPE: 17.46, MAE: 15.71
**************************************************
Ticker: CAH, MAPE: 6.89, MAE: 7.41
**************************************************
Ticker: CI, MAPE: 9.81, MAE: 31.09

In [24]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(23.232248214300924, 62.758901643799355)

## Theta model

In [26]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        season_length=30
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for A
Model for A trained in 1.00 seconds
**************************************************
Training model for ABBV
Model for ABBV trained in 0.73 seconds
**************************************************
Training model for ABT
Model for ABT trained in 1.66 seconds
**************************************************
Training model for ALGN
Model for ALGN trained in 0.71 seconds
**************************************************
Training model for AMGN
Model for AMGN trained in 1.06 seconds
**************************************************
Training model for BAX
Model for BAX trained in 1.57 seconds
**************************************************
Training model for BDX
Model for BDX trained in 1.05 seconds
**************************************************
Training model for BIIB
Model for BIIB trained in 0.96 seconds
**************************************************
Training model for BMY
Model for BMY trained in 0.79 seconds
***************************************

In [28]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: A, MAPE: 9.96, MAE: 12.08
**************************************************
Ticker: ABBV, MAPE: 10.84, MAE: 20.41
**************************************************
Ticker: ABT, MAPE: 6.15, MAE: 7.53
**************************************************
Ticker: ALGN, MAPE: 39.63, MAE: 78.84
**************************************************
Ticker: AMGN, MAPE: 8.56, MAE: 26.26
**************************************************
Ticker: BAX, MAPE: 11.55, MAE: 3.66
**************************************************
Ticker: BDX, MAPE: 6.39, MAE: 13.85
**************************************************
Ticker: BIIB, MAPE: 26.75, MAE: 39.85
**************************************************
Ticker: BMY, MAPE: 16.87, MAE: 9.10
**************************************************
Ticker: BSX, MAPE: 14.91, MAE: 13.90
**************************************************
Ticker: CAH, MAPE: 11.63, MAE: 14.11
**************************************************
Ticker: CI, MAPE: 9.43, MAE: 28.58
**

In [29]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(14.8552769107906, 33.48006276291571)

## Linear Regression

In [32]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for A
Model for A trained in 0.17 seconds
**************************************************
Training model for ABBV
Model for ABBV trained in 0.14 seconds
**************************************************
Training model for ABT
Model for ABT trained in 0.12 seconds
**************************************************
Training model for ALGN
Model for ALGN trained in 0.13 seconds
**************************************************
Training model for AMGN
Model for AMGN trained in 0.30 seconds
**************************************************
Training model for BAX
Model for BAX trained in 0.15 seconds
**************************************************
Training model for BDX
Model for BDX trained in 0.13 seconds
**************************************************
Training model for BIIB
Model for BIIB trained in 0.13 seconds
**************************************************
Training model for BMY
Model for BMY trained in 0.13 seconds
***************************************

In [33]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: A, MAPE: 3.07, MAE: 3.97
**************************************************
Ticker: ABBV, MAPE: 2.75, MAE: 4.98
**************************************************
Ticker: ABT, MAPE: 2.15, MAE: 2.56
**************************************************
Ticker: ALGN, MAPE: 29254.14, MAE: 51436.61
**************************************************
Ticker: AMGN, MAPE: 7.21, MAE: 22.08
**************************************************
Ticker: BAX, MAPE: 3.55, MAE: 1.19
**************************************************
Ticker: BDX, MAPE: 1.81, MAE: 3.95
**************************************************
Ticker: BIIB, MAPE: 6.34, MAE: 10.04
**************************************************
Ticker: BMY, MAPE: 3.52, MAE: 1.75
**************************************************
Ticker: BSX, MAPE: 2.54, MAE: 2.29
**************************************************
Ticker: CAH, MAPE: 25.41, MAE: 31.02
**************************************************
Ticker: CI, MAPE: 3.13, MAE: 9.79
******

In [34]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(26614552.579547953, 60115790.875576)